In [2]:
import os
import sys
sys.path.append('/home/kal/TF_models/bin/')
os.environ['CUDA_VISIBLE_DEVICES'] = '2' # Must be before importing keras!
import tf_memory_limit

from keras import backend as K
from keras.models import load_model, Model, Input
from keras.layers import Input, Lambda, Conv1D

import numpy as np
import matplotlib.pylab as plt 
from sklearn.metrics import precision_recall_curve
from scipy.integrate import trapz
from tqdm import tqdm
import ucscgenome
import math
import pandas as pd
import time
import importlib
import pickle

import sequence
import train_TFmodel
import eval_TFmodel
import ctcfgen
import seq_only_gen
pwm = eval_TFmodel.TFmodel('/home/kal/TF_models/seq_only/seq_classifier/pwm_frozen/')

Using TensorFlow backend.
/home/kal/TF_models/bin/sequence.py:275: RuntimeWarning: divide by zero encountered in log
  self.seq = helper.softmax(np.log(dist))


Loading model without Bias layer


In [3]:
# meta
regression=False
gen_path= '/home/kal/TF_models/seq_only/seq_regression/test_CTCF/CTCF_gen.hdf5' # give none if no generator yet
if regression:
    import train_seq_regression_convnet as train_convnet
else:
    import train_seq_classifier_convnet as train_convnet

In [4]:
if regression:
    out_path = os.path.join('/home/kal/TF_models/seq_only/seq_regression/test2_CTCF')
else:
    out_path = os.path.join('/home/kal/TF_models/seq_only/seq_classifier/test2_CTCF')
os.makedirs(out_path)
if gen_path==None:
    bed_path = '/home/kal/TF_models/data/encode_chipseq_peaks.bed'
    gen_path = os.path.join(out_path, 'CTCF_gen.hdf5')
    print('Makeing generator')
    seq_only_gen.create_from_bed(bed_path, gen_path, scrambled=2)        
print('Training model')
train_convnet.make_model(out_path, '32.3_32.32_16.3_8.3', gen_path)
ml_model = eval_TFmodel.TFmodel(out_path)

Training model
Convolutions used: [[32, 3], [32, 32], [16, 3], [8, 3]] [neurons, filter]


OSError: Unable to open file (Unable to open file: name = '/home/kal/tf_models/seq_only/seq_regression/test_ctcf/ctcf_gen.hdf5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

In [ ]:
# get training loss stuff
for file in os.listdir(os.path.join(out_path, 'history')):
    file = os.path.join(os.path.join(out_path, 'history'), file)
    # find the history pickles
    if file.endswith('1.pk1'):
        with open(file, 'rb') as infile:
            h1 = pickle.load(infile)
    elif file.endswith('2.pk1'):
        with open(file, 'rb') as infile:
            h2 = pickle.load(infile)
    elif file.endswith('3.pk1'):
        with open(file, 'rb') as infile:
            h3 = pickle.load(infile)

In [ ]:
# Summarize history for accuracy
plt.plot(eval_TFmodel.group_stats('loss', h1, h2, h3))
plt.plot(eval_TFmodel.group_stats('val_loss', h1, h2, h3))
plt.title('Training and Validation Loss for CTCF model')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# predict on all atac sequences from k562 and make a p-r and p-r gain curve
npreds = dict()
bed_path = '/home/kal/TF_models/data/K562_atac_peaks/final_atac.bed'
peaks = pd.read_table(bed_path, header=None)
peaks = peaks.sample(10000)
peaks.columns = 'chr start end ctcf_label . . '.split()
peaks = peaks[peaks['chr']!='chrM']
print(len(peaks))
pwm_preds = pwm.predict_bed(peaks)
true_labels = peaks['ctcf_label']
ml_preds = ml_model.predict_bed(peaks)

In [ ]:
#p-r curve
plt.figure(figsize=(8,6))
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('P-R Curve for CTCF Binding in K562 ATAC Regions')
plt.xlabel('Recall')
plt.ylabel('Precision')
pwm_p, pwm_r, pwm_t = precision_recall_curve(peaks['ctcf_label'], pwm_preds, pos_label=1)
plt.plot(pwm_r, pwm_p, label='PWM Model')
p, r, t = precision_recall_curve(peaks['ctcf_label'], ml_preds, pos_label=1)
plt.plot(r, p, label='ML Model')
plt.legend()
plt.show()
print('AOC: ' + str(- trapz(p,r)))